From https://pytorch.org/docs/0.3.1/nn.html?highlight=crossentropy#bcewithlogitsloss

This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability.

The loss can be described as:
ℓ(x,y)=L={l1,…,lN}⊤,ln=−wn[tn⋅logσ(xn)+(1−tn)⋅log(1−σ(xn))],

where N

is the batch size. If reduce is True, then
ℓ(x,y)={mean(L),sum(L),ifsize_average=True,ifsize_average=False.

This is used for measuring the error of a reconstruction in for example an auto-encoder. Note that the targets t[i] should be numbers between 0 and 1.
Parameters:	

    weight (Tensor, optional) – a manual rescaling weight given to the loss of each batch element. If given, has to be a Tensor of size “nbatch”.
    size_average – By default, the losses are averaged over observations for each minibatch. However, if the field size_average is set to False, the losses are instead summed for each minibatch. Default: True



In [1]:
from Where import *

SyntaxError: invalid syntax (Where.py, line 238)

## Réseau de neurones

### Lancer l'apprentissage ou charger les données

In [ ]:
path = '2018-05-23_classification_BCELoss.pt'

In [ ]:
!ls -l {path}
!rm {path}

In [ ]:
if os.path.isfile(path):
    net.load_state_dict(torch.load(path))
    print('Loading file', path)
else:
    print('Training model...')
    N_epochs = 30
    for epoch in range(N_epochs):          #max number of training epochs
        net = train(net, minibatch_size)                 #starting the learning
        torch.save(net.state_dict(), path) #save the neural network state
        print('Model saved at', path)

### Lancer l'évaluation

In [ ]:
N_test = 10

for _ in range(N_test):
    eval_sacc()
    plt.show()    

In [ ]:
for _ in range(N_test):
    eval_sacc(fig_type='log')
    plt.show()    